In [1]:
'''
匯入套件
'''

# 操作 browser 的 API
from selenium import webdriver
from selenium.webdriver.chrome.service import Service

# 處理逾時例外的工具
from selenium.common.exceptions import TimeoutException

# 面對動態網頁，等待某個元素出現的工具，通常與 exptected_conditions 搭配
from selenium.webdriver.support.ui import WebDriverWait

# 搭配 WebDriverWait 使用，對元素狀態的一種期待條件，若條件發生，則等待結束，往下一行執行
from selenium.webdriver.support import expected_conditions as EC

# 期待元素出現要透過什麼方式指定，通常與 EC、WebDriverWait 一起使用
from selenium.webdriver.common.by import By

# 強制等待 (執行期間休息一下)
from time import sleep

#美麗湯
from bs4 import BeautifulSoup as bs

# 整理 json 使用的工具
import json

#正規表達式
import re

import pandas as pd


# 由於klook 會擋 嘗試更換user-agent
user_agent = "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"

# 啟動瀏覽器工具的選項
my_options = webdriver.ChromeOptions()
# my_options.add_argument("--headless")                #不開啟實體瀏覽器背景執行
my_options.add_argument("--start-maximized")         #最大化視窗
my_options.add_argument("--incognito")               #開啟無痕模式
my_options.add_argument("--disable-popup-blocking") #禁用彈出攔截
my_options.add_argument("--disable-notifications")  #取消 chrome 推播通知
my_options.add_argument("--lang=zh-TW")  #設定為正體中文
my_options.add_argument(f"user-agent={user_agent}")

 # 加入其他可能有用的隱私設定
my_options.add_argument('--disable-blink-features=AutomationControlled')
my_options.add_argument('--disable-extensions')
my_options.add_experimental_option('excludeSwitches', ['enable-automation'])
my_options.add_experimental_option('useAutomationExtension', False)
# 使用 Chrome 的 WebDriver
driver = webdriver.Chrome(
    options = my_options
)

In [6]:
'''
以 function 名稱，作為爬蟲流程
'''


# 設定景點名稱 klook的編號 爬取起始頁面
title = '101觀景台'
activity_id = '1659'

# 放置爬取的資料
listData = []

# 清空儲存列表
def clear_list():
     # 使用全域變數
    global listData
    # 清空列表
    listData.clear()

def get_content(target=20):
    global listData
    
    # 前往klook 頁面 單一景點評論上限2000筆 單頁get限制100筆
    page = 1
    
    while page <= target:
        url = f'https://www.klook.com/v1/experiencesrv/activity/component_service/activity_reviews_list?k_lang=zh_TW&k_currency=TWD&activity_id={activity_id}&page={page}&limit=100&star_num=&lang=&sort_type=0&only_image=false'
        driver.get(url)
        try:   
            # 等待內文載入
            WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.CSS_SELECTOR, "pre"))
            )
            
            # 取得內文並轉為json物件
            obj = json.loads(driver.find_element(By.CSS_SELECTOR, "pre").get_attribute("innerText"))

            # 輸出目前爬取頁數
            print(f'目前爬取頁數:{obj["result"]["current_page"]}')

            # 將評論內容放入list
            for o in obj['result']['item']:
                listData.append({
                        'sources':"Klook",
                        'title': title,
                        'id':o['id'],
                        'time':o['date'],
                        'rating':o['rating'],
                        'content': o['content'],
                        'translate_content':o['translate_content']
                    })
            # 若無法載入則顯示錯誤       
        except TimeoutException:
            print('timeout error')
        page += 1
        sleep(3)

def to_csv():
    # 將結果轉為data frame
    df = pd.DataFrame(listData)
    # 取得文章數量
    print(len(df))
    # 輸出為csv檔
    df.to_csv(f'klook_{title}.csv', index=True, encoding='utf-8-sig')

def close():   
    driver.quit()


In [7]:
if __name__ == '__main__':
    clear_list()
    get_content()
    to_csv()

目前爬取頁數:1
目前爬取頁數:2
目前爬取頁數:3
目前爬取頁數:4
目前爬取頁數:5
目前爬取頁數:6
目前爬取頁數:7
目前爬取頁數:8
目前爬取頁數:9
目前爬取頁數:10
目前爬取頁數:11
目前爬取頁數:12
目前爬取頁數:13
目前爬取頁數:14
目前爬取頁數:15
目前爬取頁數:16
目前爬取頁數:17
目前爬取頁數:18
目前爬取頁數:19
目前爬取頁數:20
2000


In [9]:
# 等確定擷取流程結束後，再手動關閉瀏覽器，以便 debug，減少瀏覽器開開關關
close()